# 2) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN data set.

# Import statements
Following packages are necessary:

In [9]:
import random
import utils

import tensorflow as tf
import numpy as np
import segmentation_models as sm

from unet import UNet
from unet import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from lastengen import LASTENSequence

# Checks
The version of tensorflow as well as the GPU support are checked.

In [10]:
print(tf.__version__)
tf.config.experimental.list_physical_devices('GPU')

2.2.0-rc3


[]

# Data
Import training images

In [ ]:
path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/train'
path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'
height=768
width=768

# Seeding
Seeds are set to ensure reproducible training.

In [ ]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [ ]:
model = UNet(input_shape=(height, width, 1))
model.summary()

# Data Preparation
The generator sequence for training is built.

In [ ]:
width = 384
height = 384

train_gen = LASTENSequence(path_train,
                           batch_size=8,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=8,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

test_gen = LASTENSequence(path_train,
                          batch_size=8,
                          preprocess_input=pre_une,
                          width=width,
                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [ ]:
logger = CSVLogger("experiments/segmentation.log")
checker = ModelCheckpoint("experiments/segmentation/weights.{epoch:02d}.hdf5",
                          period=1)

callbacks = [logger, checker]

model.compile(optimizer='adam',
              loss = {'segmentation': dice_loss},
              metrics = {'segmentation': ['accuracy', iou_score]})

# Training
The model is fitted with a generator.

In [ ]:
model.fit_generator(train_gen,
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=train_gen,
                    validation_freq=1,
                    use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)